<a href="https://colab.research.google.com/github/Sayandeep27/Natural-Language-Processing/blob/main/PDF_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fitz nltk transformers tensorflow flask pickle5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255318 sha256=f2c5e2c2e520abcb829531b74e3b13c177efb2bc97d2dfa7ef600037e82e2cf7
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [ ]:
pip install PyMuPDF

In [ ]:
import fitz  # PyMuPDF
import nltk
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import tensorflow as tf
import pickle

# Download NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Define the functions for the summarizer
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

def preprocess_text(text):
    sentences = nltk.sent_tokenize(text)
    return " ".join(sentences)

def summarize_text(text, max_length=150, min_length=50):
    tokenizer = AutoTokenizer.from_pretrained('t5-small')
    model = TFAutoModelForSeq2SeqLM.from_pretrained('t5-small')
    inputs = tokenizer.encode("summarize: " + text, return_tensors='tf', max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
# Save the tokenizer and model using pickle
tokenizer = AutoTokenizer.from_pretrained('t5-small')
model = TFAutoModelForSeq2SeqLM.from_pretrained('t5-small')

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('model.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
